In [25]:
%pip install selenium
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Credenciales de Twitter
username = 'ScrapeoUPM'
password = 'scrapeafutbol'

SCROLL_PAUSE_TIME = 2  # Tiempo de espera para que carguen nuevos tweets

# Iniciar el navegador con Selenium y WebDriver Manager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Navegar a la página de inicio de sesión de Twitter
driver.get('https://twitter.com/login')
time.sleep(10)  # Esperar a que cargue la página

# Buscar el campo del nombre de usuario e ingresar el nombre de usuario o correo
username_input = driver.find_element(By.NAME, 'text')
username_input.send_keys(username)
driver.find_element(By.XPATH, '//*[@id="layers"]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/button[2]').click()
time.sleep(2)

# Buscar el campo de contraseña e ingresar la contraseña
password_input = driver.find_element(By.NAME, 'password')
password_input.send_keys(password)

# Hacer clic en el botón de iniciar sesión
driver.find_element(By.XPATH, '//*[@id="layers"]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/button').click()
time.sleep(5)  # Esperar unos segundos para asegurarse de que el login se ha realizado

# Ahora que has iniciado sesión, puedes navegar a Twitter y hacer scraping
driver.get('https://twitter.com/search?q=Python')

# Esperar que los tweets carguen
time.sleep(5)

# Extraer tweets (esto puede necesitar ajustes dependiendo de la estructura de la página)
tweets = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="tweetText"]')

for tweet in tweets:
    print(tweet.text)

# Scroll y extraer tweets
for i in range(5):  # Ajusta el rango según la cantidad de scrolls que desees hacer
    # Hacer scroll hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Esperar que carguen nuevos tweets
    time.sleep(SCROLL_PAUSE_TIME)

    # Extraer los tweets visibles en este momento
    tweets = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="tweetText"]')

    # Imprimir el texto de los tweets extraídos
    for tweet in tweets:
        print(tweet.text)

# Cerrar el navegador
driver.quit()

Barcode using Python
Python Projects for Hacking
Yes, I am 52. 

Yes, I am learning Python  

No, you are never too old to learn how to code.
“Wednesday”‘s Jenna Ortega reveals in an interview to Vogue that her favorite programming language is Python.

“I really enjoy writing Python.. It a lot easier than JavaScript - I love how readable it is. It’s so robust.”
Python, Falcon and the Sea 
Python basics and syntax.
Machine Learning Books for #ComputerVision. #BigData #Analytics #DataScience #IoT #IIoT #Python #RStats #TensorFlow #Java #JavaScript #ReactJS #GoLang #CloudComputing #Serverless #DataScientist #Linux #Books #Programming #Coding #100DaysofCode 
https://geni.us/M-L-Books-CompVision…
Bubble Sort
#PyConUS2022 
@pwang
 Keynote: Announcing Py-script!!!
It's Python! inside HTML!!! 
C programmers watching Python programmers work
Appearance of a python’s teeth
𝐀𝐥𝐥 𝐏𝐚𝐢𝐝 𝐂𝐨𝐮𝐫𝐬𝐞𝐬 (𝐅𝐫𝐞𝐞 𝐟𝐨𝐫 𝐅𝐢𝐫𝐬𝐭 𝟓𝟎𝟎𝟎 𝐏𝐞𝐨𝐩𝐥𝐞)

1. Artificial Intelligence
2. Machine Learning
3. Cloud Computing
4. Ethical H